In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print ("TF version : [{}]".format(tf.__version__))

/Users/noah/venv_study/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


TF version : [1.4.0]


In [2]:
mnist = input_data.read_data_sets('mnist_data/', one_hot=True)

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [3]:
# network topology
n_hidden_1 = 256
n_hidden_2 = 128
n_input = 784
n_classes = 10

# inputs and outputs
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# network parameter
stddev = 0.1
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("network ready")

network ready


In [4]:
# model
# softmax를 취하지 않은 로짓을 리턴
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, _weights['h1']), _biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    logit = tf.add(tf.matmul(layer_2, _weights['out']), _biases['out'])
    return logit

# prediction
pred = multilayer_perceptron(x, weights, biases)

# loss and optimizer
cost0 = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)    # 각 샘플별 cross entropy 값
cost = tf.reduce_mean(cost0)
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
# tf.argmax(pred, 1): row 방향에서 argmax를 취한다
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, tf.float32))

# initializer
init = tf.global_variables_initializer()
print ("function ready")

function ready


In [8]:
# parameters
training_epochs = 20
batch_size = 100
display_step = 4

# launch the graph
sess = tf.Session()
sess.run(init)

# optimize
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)

    # iteration
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})
    avg_cost = avg_cost/total_batch
    
    # display
    if (epoch+1) % display_step == 0:
        print ("epoch: %03d/%03d  cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
        print ("train accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print ("test accuracy: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

epoch: 003/020  cost: 0.118304569
train accuracy: 0.970
test accuracy: 0.962
epoch: 007/020  cost: 0.047794442
train accuracy: 0.980
test accuracy: 0.975
epoch: 011/020  cost: 0.019027545
train accuracy: 0.990
test accuracy: 0.974
epoch: 015/020  cost: 0.007002827
train accuracy: 0.990
test accuracy: 0.977
epoch: 019/020  cost: 0.002356627
train accuracy: 1.000
test accuracy: 0.981
OPTIMIZATION FINISHED
